In [ ]:
import meep as mp
import math
import cmath


# Cube resonator analysis

# default unit length is 1 um
um_scale = 1.0


    
resolution = 40     # pixels/um

a = 3.4       # lattice periodicity
d = 1.7        # cube side length
#h = 0.2             # metal rod height

tcell= 1.7     # unitcell thickness    
tsub = 4.85          # substrate thickness
tpml = 5.0          # PML thickness
tair = 4.0          # air thickness

sz = 2*tpml+tair+tcell+tsub
cell_size = mp.Vector3(a,a,sz)

pml_layers = [mp.PML(thickness=tpml,direction=mp.Z)]


lmin = 5.0         # source min wavelength
lmax = 10.0         # source max wavelength
fmin = 1/lmax       # source min frequency
fmax = 1/lmin       # source max frequency
fcen = 0.5*(fmin+fmax)
df = fmax-fmin

# Generate 
nTe = 5.3
Te = mp.Medium(index=nTe)

nbaf2=1.4
BaF2=mp.Medium(index=nbaf2)


geometry = [mp.Block(material=Te, size=mp.Vector3(d,d,d),
                     center=mp.Vector3(0,0,0)),
                     mp.Block(material=BaF2, size=mp.Vector3(mp.inf,mp.inf,tsub+tpml),
                              center=mp.Vector3(0,0,-tsub/2.0-d/2.0)) ]

# CCW rotation angle (degrees) about Y-axis of PW current source; 0 degrees along -z axis

theta=0 # make it constant
theta = math.radians(theta)

# k with correct length (plane of incidence: XZ) 
k = mp.Vector3(math.sin(theta),0,math.cos(theta)).scale(fcen)

def pw_amp(k, x0):
    def _pw_amp(x):
        return cmath.exp(1j * 2 * math.pi * k.dot(x + x0))
    return _pw_amp

src_pos = d/2.0+tair*0.75
sources = [ mp.Source(mp.GaussianSource(fcen, fwidth=df), component=mp.Ey, center=mp.Vector3(0,0,src_pos),
                          size=mp.Vector3(a,a,0),
                          amp_func=pw_amp(k, mp.Vector3(0,0,src_pos))) ]

sim = mp.Simulation(cell_size=cell_size,
                        geometry=geometry,
                        sources=sources,
                        boundary_layers=pml_layers,
                        k_point = k,
                        resolution=resolution)

nfreq = 50
refl = sim.add_flux(fcen, df, nfreq, mp.FluxRegion(center=mp.Vector3(0,0,d/2.0+0.5*tair),size=mp.Vector3(a,a,0)))


sim.load_minus_flux('refl-flux', refl)
    
sim.run(until_after_sources=mp.stop_when_fields_decayed(25, mp.Ey, mp.Vector3(0,0,d/2.0+0.5*tair), 1e-7))


sim.save_flux('refl-flux', refl)
    
sim.display_fluxes(refl)



-----------
Initializing structure...
Working in 3D dimensions.
Computational cell is 3.4 x 3.4 x 20.55 with resolution 40
     block, center = (0,0,0)
          size (1.7,1.7,1.7)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (28.09,28.09,28.09)
     block, center = (0,0,-3.275)
          size (1e+20,1e+20,9.85)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1.96,1.96,1.96)
subpixel-averaging is 91.7881% done, 0.357913 s remaining
subpixel-averaging is 43.4334% done, 5.21022 s remaining
subpixel-averaging is 86.8795% done, 0.604181 s remaining
subpixel-averaging is 92.3684% done, 0.33055 s remaining
subpixel-averaging is 43.4587% done, 5.20508 s remaining
subpixel-averaging is 86.9934% done, 0.598091 s remaining
subpixel-averaging is 94.5049% done, 0.232655 s remaining
subpixel-averaging is 90.3375% done, 0.42786 s remaining
subpixel-averaging is 43.3954% done, 5.21844 s remaining
subpixel-averagin